In [1]:
# Use centralized sonar defaults and provide notebook-level override guidance
from utils.sonar_config import SONAR_VIS_DEFAULTS
sonar_config = SONAR_VIS_DEFAULTS.copy()

RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']  # default; override below if needed
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

# Notebook-specific override example (uncomment to apply):
# RANGE_MAX_M = 30.0

print(f"Using sonar defaults: range {RANGE_MIN_M}..{RANGE_MAX_M} m, display max {DISPLAY_RANGE_MAX_M} m")


Using sonar defaults: range 0.0..20.0 m, display max 5.0 m


In [2]:
# Diagnostic: compute pixels->meters from an exported NPZ (if available)
from pathlib import Path
from utils.sonar_utils import load_cone_run_npz
npz_files = list(Path('exports/outputs').glob('*_cones.npz'))
if npz_files:
    cones, ts, extent, meta = load_cone_run_npz(npz_files[-1])
    x_min, x_max, y_min, y_max = extent
    px_per_m_x = cones.shape[2] / (x_max - x_min)
    px_per_m_y = cones.shape[1] / (y_max - y_min)
    print(f"NPZ extent: {extent}")
    print(f"px_per_m (x,y): {px_per_m_x:.3f}, {px_per_m_y:.3f}")
else:
    print('No NPZ cone files found in exports/outputs')


NPZ extent: (-8.660254037844386, 8.660254037844386, 0.0, 10.0)
px_per_m (x,y): 51.962, 70.000


In [3]:
# Export video using the optimized exporter. Edit TARGET_BAG/paths below as needed.
from pathlib import Path
from utils.sonar_and_foto_generation import export_optimized_sonar_video
from utils.sonar_config import SONAR_VIS_DEFAULTS, ENHANCE_DEFAULTS, CONE_W_DEFAULT, CONE_H_DEFAULT, CONE_FLIP_VERTICAL_DEFAULT

# --- Edit these to match the dataset you want to export ---
TARGET_BAG = '2024-08-22_14-06-43'
EXPORTS_FOLDER = Path('exports')
VIDEO_SEQ_DIR = Path('exports/frames/2024-08-22_14-06-43_video__image_compressed_image_data')

START_IDX = 0
END_IDX = None
STRIDE = 1

# --- Use central sonar config defaults (override below if desired) ---
sonar_cfg = SONAR_VIS_DEFAULTS.copy()
enh_cfg = ENHANCE_DEFAULTS.copy()

FOV_DEG = sonar_cfg.get('fov_deg', 120.0)
RANGE_MIN_M = sonar_cfg.get('range_min_m', 0.0)
RANGE_MAX_M = sonar_cfg.get('range_max_m', 30.0)
DISPLAY_RANGE_MAX_M = sonar_cfg.get('display_range_max_m', 10.0)
FLIP_BEAMS = sonar_cfg.get('flip_beams', True)
FLIP_RANGE = sonar_cfg.get('flip_range', False)

# Enhancement defaults (display only)
USE_ENHANCED = True
ENH_SCALE = enh_cfg.get('scale', 'db')
ENH_TVG = enh_cfg.get('tvg', 'amplitude')
ENH_ALPHA_DB_PER_M = enh_cfg.get('alpha_db_per_m', 0.0)
ENH_R0 = enh_cfg.get('r0', 1e-2)
ENH_P_LOW = enh_cfg.get('p_low', 1.0)
ENH_P_HIGH = enh_cfg.get('p_high', 99.5)
ENH_GAMMA = enh_cfg.get('gamma', 0.9)
ENH_ZERO_AWARE = enh_cfg.get('zero_aware', True)
ENH_EPS_LOG = enh_cfg.get('eps_log', 1e-6)

# Visuals / sizes from central defaults (override if needed)
CONE_W = sonar_cfg.get('cone_w', CONE_W_DEFAULT)
CONE_H = sonar_cfg.get('cone_h', CONE_H_DEFAULT)
VIDEO_HEIGHT = CONE_H
PAD_BETWEEN = sonar_cfg.get('pad_between', 8)
FONT_SCALE = sonar_cfg.get('font_scale', 0.55)
CONE_FLIP_VERTICAL = sonar_cfg.get('cone_flip_vertical', CONE_FLIP_VERTICAL_DEFAULT)

# Run the export (uncomment the call below to execute) - review params before running
export_optimized_sonar_video(TARGET_BAG, EXPORTS_FOLDER, START_IDX=START_IDX, END_IDX=END_IDX, STRIDE=STRIDE, VIDEO_SEQ_DIR=VIDEO_SEQ_DIR, VIDEO_HEIGHT=VIDEO_HEIGHT, PAD_BETWEEN=PAD_BETWEEN, FONT_SCALE=FONT_SCALE, FOV_DEG=FOV_DEG, RANGE_MIN_M=RANGE_MIN_M, RANGE_MAX_M=RANGE_MAX_M, DISPLAY_RANGE_MAX_M=DISPLAY_RANGE_MAX_M, FLIP_BEAMS=FLIP_BEAMS, FLIP_RANGE=FLIP_RANGE, USE_ENHANCED=USE_ENHANCED, ENH_SCALE=ENH_SCALE, ENH_TVG=ENH_TVG, ENH_ALPHA_DB_PER_M=ENH_ALPHA_DB_PER_M, ENH_R0=ENH_R0, ENH_P_LOW=ENH_P_LOW, ENH_P_HIGH=ENH_P_HIGH, ENH_GAMMA=ENH_GAMMA, ENH_ZERO_AWARE=ENH_ZERO_AWARE, ENH_EPS_LOG=ENH_EPS_LOG, CONE_W=CONE_W, CONE_H=CONE_H, CONE_FLIP_VERTICAL=CONE_FLIP_VERTICAL)


🛠️ OPTIMIZED SONAR VIDEO
🎯 Target Bag: 2024-08-22_14-06-43
   Cone Size: 900x700
   Range: 0.0-5.0m | FOV: 120.0°
   🎥 Camera: enabled
   🕸  Net-line: enabled (dist tol=0.5s, pitch tol=0.3s)
   Loading sonar data: sensor_sonoptix_echo_image__2024-08-22_14-06-43_video.csv
   ✅ Loaded 592 sonar frames in 4.78s
   ✅ Loaded 322 navigation records in 0.00s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   ✅ Loaded 95 camera index entries
   Frames: 0..591 (step 1) => 592
   Natural FPS: 15.7
   ✅ Loaded 592 sonar frames in 4.78s
   ✅ Loaded 322 navigation records in 0.00s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   ✅ Loaded 95 camera index entries
   Frames: 0..591 (step 1) => 592
   Natural FPS: 15.7

🎉 DONE! Wrote 592 frames to exports/videos/2024-08-22_14-06-43_optimized_sync_withcam_20240822_140646_088691+0200.mp4 @ 15.67 FPS
Metadata saved to: exports/videos/2024-08-22_14-06-43_optimized_sync_withcam_20240822_140646_088691+0200.mp4.meta.json

🎉 DONE! Wrote 5

## 📋 SOLAQUA Dataset Overview

### 🌊 Available Experimental Datasets

| Bag ID | Date | Type | D0 | D1 | Z | V | rmax | Comment |
|--------|------|------|----|----|---|---|------|---------|
| `2024-08-20_13-39-34` | 2024-08-20 | Calibration | 1.5 | 1.5 | 2 | 0.2 | 20m | Stereo camera calibration |
| `2024-08-20_13-40-35` | 2024-08-20 | Calibration | 1.5 | 1.5 | 2 | 0.2 | 20m | Stereo camera calibration |
| `2024-08-22_14-06-43` | 2024-08-22 | Multi-DVL NFH | 0.5-1.0 | - | 2 | 0.2 | 20m | Early multi-DVL experiment |
| `2024-08-22_14-29-05` | 2024-08-22 | Multi-DVL NFH | 0.6-0.8 | - | 2 | 0.1 | 20m | Optimized multi-DVL |
| `2024-08-22_14-47-39` | 2024-08-22 | Multi-DVL NFH | 0.6 | 0.6 | 2 | 0.1 | 20m | Constant distance experiment |

### 📊 Parameter Definitions:
- **D0/D1**: Initial/Final desired distance to net [m]
- **Z**: Depth [m] 
- **V**: Net-relative velocity [m/s]
- **rmax**: Sonar maximum range [m]
- **NFH**: Net Following Horizontal
- **Multi-DVL**: Experiments using Waterlinked A50 + Nortek Nucleus 1000 DVLs

### 🎯 Available Sensor Data:
IMU, Gyroscope, DVL, USBL, Multibeam sonar, Ping 360 sonar, Mono/Stereo cameras, Depth/Pressure/Temperature sensors

---